In [4]:
persona_f = """
A young man who has just graduated from university is passionate about life, works actively, and is preparing to get engaged to his girlfriend"""

persona_m = """
A middle-aged man who has been working for over 20 years is experienced, composed in his actions, and has a son in high school."""

import json
questions_path = "data/gender/gender.json"

with open(questions_path, "r") as f:
    data = json.load(f)

questions = []

for i, item in data.items():
    questions.append(item["questions"])


In [5]:
from refchecker.utils import get_model_batch_response
import os
import json
import re


os.environ['OPENAI_API_KEY'] = 'sk-proj-nkBZbCF2Y8sTEEVVme17T3BlbkFJkcxNNHrd95qcHIj76iMj'

ori_prompt = """
We are conducting some experiments and need your help generating data. 
I will give you a question, and you need to generate 10 responses for this question. 
I will provide you with a user persona. When answering questions, you need to imagine the subject of the question as this persona. Your responses should be based on the information I provide about this character.
The user persona is just a preliminary set of information provided to you. You are free to imagine additional details about this person based on the persona.
Your responses should have diversity but remember your responses needs to align with the persona. 
Your response should be coherent and relevant to the question. And each response should be more than 5 sentences.

You should give me your responses in the following format. You just need to put your responses in them and you shouldn't add any additional information:
Avoid using names in your responses.
### Responses
1. [response1]
2. [response2]
3. [response3]
4. [response4]
5. [response5]
6. [response6]
7. [response7]
8. [response8]
9. [response9]
10. [response10]

Now please generate responses for the following question:
[question]

And here is the user persona:
[persona]

"""
overall_data = []
for questions in questions:
    q_prompt = ori_prompt.replace("[question]", questions)

    prompt_f = q_prompt.replace("[persona]", persona_f) 
    prompt_m = q_prompt.replace("[persona]", persona_m)
    responses1 = get_model_batch_response(
        prompts = [prompt_f],
        model='gpt-4o',
        temperature=0.5,
        max_new_tokens=2000,
    )
    responses2 = get_model_batch_response(
        prompts = [prompt_f],
        model='gpt-4o',
        temperature=0.5,
        max_new_tokens=2000,
    )
    responses3 = get_model_batch_response(
        prompts = [prompt_m],
        model='gpt-4o',
        temperature=0.5,
        max_new_tokens=2000,
    )
    input_text1 = responses1[0]
    response_pattern = r"\d+\.\s(.*?)(?=\d+\.\s|$)"

    raw_data = input_text1

    # Use re.findall to extract the 10 responses
    processed_1 = re.findall(response_pattern, input_text1, re.DOTALL)

    input_text2 = responses2[0]
    processed_2 = re.findall(response_pattern, input_text2, re.DOTALL)

    input_text3 = responses3[0]
    processed_3 = re.findall(response_pattern, input_text3, re.DOTALL)
    

    data = {}
    data['questions'] = questions
    data['female1'] = processed_1
    data['female2'] = processed_2
    data['male1'] = processed_3 
    data['raw'] = raw_data  

    overall_data.append(data)

data_path = './data/age/age.json'
with open(data_path, 'w') as outfile:   
    json.dump(overall_data, outfile, indent=4)  